Attempt to predict wave heights with historic met data.<ol>
<li> 1. Predict single location based on wind, pressure, seasonality</li>
<li> 2. Predict wave height based on wave height of other buoys</li>
</ol>

<p> Data is from Coastal observatory (waves) and CEDA (weather)</p>
ob_time	wind_direction	wind_speed	msl_pressure	air_temperature



In [3]:
#reading in unprocessed data
import glob as glob
import pandas as pd
import numpy as np
from pathlib import Path
#visialisation
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
# Importing weather data
glob_path = Path(r"C:\Users\rhnae\Downloads\data\waveHeightPrediction")
file_list = [str(pp) for pp in glob_path.glob("*.csv")]
file_list
wind_data = []
for fileIn in file_list:
    #print(fileIn)
    df = pd.read_csv(fileIn, 
    header = 280,
    usecols = ['ob_time','wind_direction','wind_speed','msl_pressure', 'air_temperature'],
    skipfooter = 1)
    wind_data.append(df)
wind_data = pd.concat(wind_data)
wind_data['ob_time'] = pd.to_datetime(wind_data['ob_time'])
#
wind_data.reset_index(drop = True, inplace = True)
def seasonality(df):
    from time import strftime
    """
    Returns data according to season 1. Winter, 2. Spring, 3. Summer or 4. Autumn
    """
    seasons = {
        'Winter': ['December', 'January','February'],
        'Spring': ['March', 'April','May'],
        'Summer': ['June','July','August'],
        'Autumn': ['September', 'October','November' ]}
    
    indices = [None]*len(df)

    for i in range(len(df)):
        for idx, season in enumerate(seasons):
            if df.ob_time[i].strftime('%B') in seasons[season]:
                indices[i] = season  
    return indices  

wind_data['season'] = seasonality(wind_data) 
wind_data


,ob_time,wind_direction,wind_speed,msl_pressure,air_temperature,season
0,2010-01-01 00:00:00,50.0,8.0,1003.9,2.1,Winter
1,2010-01-01 01:00:00,50.0,9.0,1004.0,2.1,Winter
2,2010-01-01 02:00:00,40.0,8.0,1004.2,1.4,Winter
3,2010-01-01 03:00:00,40.0,5.0,1004.6,1.2,Winter
4,2010-01-01 04:00:00,80.0,1.0,1004.9,-1.0,Winter
...,...,...,...,...,...,...
78806,2018-12-31 19:00:00,300.0,9.0,1037.1,9.7,Winter
78807,2018-12-31 20:00:00,280.0,11.0,1037.2,9.7,Winter
78808,2018-12-31 21:00:00,300.0,5.0,1037.1,9.5,Winter
78809,2018-12-31 22:00:00,300.0,11.0,1037.1,9.6,Winter


In [5]:
# import wave data

glob_path = Path(r"C:\Users\rhnae\Downloads\data\waveHeightPrediction\Bideford Bay_qc_data")
file_list = [str(pp) for pp in glob_path.glob("*.txt")]
file_list
wave_data = []
for fileIn in file_list:
    #print(fileIn)
    df = pd.read_csv(fileIn,sep='\t')
    wave_data.append(df)
wave_data = pd.concat(wave_data)
wave_data['ob_time'] = pd.to_datetime(wave_data['Date/Time (GMT)'])



In [6]:
wave_data.drop(['Date/Time (GMT)'],axis = 1, inplace = True)
# wave_data.reset_index(drop = True, inplace = True)
wave_data

,Latitude,Longitude,Flag,Hs (Hm0)(m),Hmax (m),Tp (s),Tz (Tm)(s),Dirp (degrees),Spread (deg),SST (deg C),ob_time
0,51.05787,-4.27703,0,0.49,0.69,10.5,3.9,291,17,8.4,2010-01-01 00:00:00
1,51.05787,-4.27703,0,0.50,0.81,11.8,4.2,293,14,8.5,2010-01-01 00:30:00
2,51.05787,-4.27703,0,0.53,0.74,12.5,4.0,281,17,8.6,2010-01-01 01:00:00
3,51.05785,-4.27705,0,0.58,0.83,4.2,3.7,352,18,8.6,2010-01-01 01:30:00
4,51.05806,-4.27716,0,0.60,0.95,4.5,3.5,345,18,8.6,2010-01-01 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...
1435,51.05834,-4.27662,0,0.61,0.91,2.9,3.0,263,18,15.2,2020-06-30 21:30:00
1436,51.05835,-4.27664,0,0.58,0.78,4.8,3.0,281,19,15.2,2020-06-30 22:00:00
1437,51.05834,-4.27660,0,0.60,0.79,5.0,3.0,283,23,15.1,2020-06-30 22:30:00
1438,51.05834,-4.27658,0,0.64,0.99,4.8,3.1,274,18,15.0,2020-06-30 23:00:00


In [89]:
#merge dataframes
all_data = pd.merge_asof(wind_data,wave_data, on='ob_time' )
all_data.set_index('ob_time', inplace = True) # eeded for interpolation later
all_data.head(2)

,wind_direction,wind_speed,msl_pressure,air_temperature,season,Latitude,Longitude,Flag,Hs (Hm0)(m),Hmax (m),Tp (s),Tz (Tm)(s),Dirp (degrees),Spread (deg),SST (deg C)
ob_time,,,,,,,,,,,,,,,
2010-01-01 00:00:00,50.0,8.0,1003.9,2.1,Winter,51.05787,-4.27703,0,0.49,0.69,10.5,3.9,291,17,8.4
2010-01-01 01:00:00,50.0,9.0,1004.0,2.1,Winter,51.05787,-4.27703,0,0.53,0.74,12.5,4.0,281,17,8.6


In [90]:
# Need to remove flagged data - chosen to interpolate
#all_data = all_data.fillna(9999).interpolate()
all_data.replace(to_replace = 9999, value = np.nan,inplace = True)

list1 = ['Hs (Hm0)(m)','Hmax (m)','Tp (s)', 'Tz (Tm)(s)', 'Dirp (degrees)', 'Spread (deg)']
# all_data.replace(9999,'NaN',inplace = True)
for col in all_data:
    if col in list1:
       all_data.loc[(all_data['Flag']> 0),col]  = np.nan
   # all_data.loc[col,9999] = np.nan # replace with nan first
    all_data[col].interpolate(method='time',inplace=True)


all_data.reset_index(drop = False, inplace = True)
all_data 

,ob_time,wind_direction,wind_speed,msl_pressure,air_temperature,season,Latitude,Longitude,Flag,Hs (Hm0)(m),Hmax (m),Tp (s),Tz (Tm)(s),Dirp (degrees),Spread (deg),SST (deg C)
0,2010-01-01 00:00:00,50.0,8.0,1003.9,2.1,Winter,51.05787,-4.27703,0,0.49,0.69,10.5,3.9,291.0,17.0,8.4
1,2010-01-01 01:00:00,50.0,9.0,1004.0,2.1,Winter,51.05787,-4.27703,0,0.53,0.74,12.5,4.0,281.0,17.0,8.6
2,2010-01-01 02:00:00,40.0,8.0,1004.2,1.4,Winter,51.05806,-4.27716,0,0.60,0.95,4.5,3.5,345.0,18.0,8.6
3,2010-01-01 03:00:00,40.0,5.0,1004.6,1.2,Winter,51.05853,-4.27626,0,0.64,0.96,5.3,4.0,353.0,19.0,8.7
4,2010-01-01 04:00:00,80.0,1.0,1004.9,-1.0,Winter,51.05848,-4.27589,0,0.61,1.14,5.0,3.9,347.0,24.0,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78806,2018-12-31 19:00:00,300.0,9.0,1037.1,9.7,Winter,51.05769,-4.27699,0,1.22,1.82,10.0,6.2,273.0,10.0,10.2
78807,2018-12-31 20:00:00,280.0,11.0,1037.2,9.7,Winter,51.05768,-4.27662,0,1.32,2.06,10.0,6.5,278.0,8.0,10.3
78808,2018-12-31 21:00:00,300.0,5.0,1037.1,9.5,Winter,51.05793,-4.27608,0,1.42,2.06,10.5,6.6,280.0,9.0,10.3
78809,2018-12-31 22:00:00,300.0,11.0,1037.1,9.6,Winter,51.05806,-4.27600,0,1.51,2.85,11.1,6.6,277.0,9.0,10.3


In [9]:
import sweetviz as sv
sweet_report = sv.analyze(all_data)
sweet_report.show_html()

:FEATURES DONE:                    |█████████████████████| [100%]   00:06  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:01  -> (00:00 left)
Creating Associations graph...DONE!
Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


In [84]:
from bokeh.plotting import figure, output_file, show
output_file("Waves.html")
f = figure(title="'Hmax Comparison", y_axis_label='Wave Height (m)')
f.line(all_data['ob_time'], all_data['Hmax (m)'])
#f.line(all_data.index.values, all_data['Hmax (m)'])
show(f)

In [85]:
# MLP with keras + sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc_df = pd.DataFrame(enc.fit_transform(all_data[['season']]).toarray())
col_names = enc.get_feature_names(['season'])
enc_df.columns = col_names

all_data = all_data.join(enc_df)
all_data.drop(['season'], axis = 1, inplace = True)
all_data


,wind_direction,wind_speed,msl_pressure,air_temperature,Latitude,Longitude,Flag,Hs (Hm0)(m),Hmax (m),Tp (s),Tz (Tm)(s),Dirp (degrees),Spread (deg),SST (deg C),season_Autumn,season_Spring,season_Summer,season_Winter
ob_time,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,50.0,8.0,1003.9,2.1,51.05787,-4.27703,0,0.49,0.69,10.5,3.9,291.0,17.0,8.4,NaN,NaN,NaN,NaN
2010-01-01 01:00:00,50.0,9.0,1004.0,2.1,51.05787,-4.27703,0,0.53,0.74,12.5,4.0,281.0,17.0,8.6,NaN,NaN,NaN,NaN
2010-01-01 02:00:00,40.0,8.0,1004.2,1.4,51.05806,-4.27716,0,0.60,0.95,4.5,3.5,345.0,18.0,8.6,NaN,NaN,NaN,NaN
2010-01-01 03:00:00,40.0,5.0,1004.6,1.2,51.05853,-4.27626,0,0.64,0.96,5.3,4.0,353.0,19.0,8.7,NaN,NaN,NaN,NaN
2010-01-01 04:00:00,80.0,1.0,1004.9,-1.0,51.05848,-4.27589,0,0.61,1.14,5.0,3.9,347.0,24.0,8.6,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,300.0,9.0,1037.1,9.7,51.05769,-4.27699,0,1.22,1.82,10.0,6.2,273.0,10.0,10.2,NaN,NaN,NaN,NaN
2018-12-31 20:00:00,280.0,11.0,1037.2,9.7,51.05768,-4.27662,0,1.32,2.06,10.0,6.5,278.0,8.0,10.3,NaN,NaN,NaN,NaN
2018-12-31 21:00:00,300.0,5.0,1037.1,9.5,51.05793,-4.27608,0,1.42,2.06,10.5,6.6,280.0,9.0,10.3,NaN,NaN,NaN,NaN


In [86]:
# split into training and testing datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense

#split data
y = np.asarray(all_data['Hs (Hm0)(m)'])
x = np.asarray(all_data.drop([['ob_time','Hs (Hm0)(m)'],axis = 1))
#x = np.asarray(x)
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2)
print('Train shape: ',x_train.shape)
print('Validation shape: ', y_val.shape)


Train shape:  (63048, 17)
Validation shape:  (15763,)


In [88]:
x_val

array([[ 190. ,    5. , 1028.2, ...,    nan,    nan,    nan],
       [ 120. ,    7. , 1002.9, ...,    nan,    nan,    nan],
       [ 110. ,   13. , 1017.7, ...,    nan,    nan,    nan],
       ...,
       [ 120. ,   17. , 1001.8, ...,    nan,    nan,    nan],
       [ 230. ,    6. , 1012.5, ...,    nan,    nan,    nan],
       [ 270. ,   11. , 1033.3, ...,    nan,    nan,    nan]])

array([[Timestamp('2010-01-01 00:00:00'), 50.0, 8.0, ..., 291.0, 17.0,
        8.4],
       [Timestamp('2010-01-01 01:00:00'), 50.0, 9.0, ..., 281.0, 17.0,
        8.6],
       [Timestamp('2010-01-01 02:00:00'), 40.0, 8.0, ..., 345.0, 18.0,
        8.6],
       ...,
       [Timestamp('2018-12-31 21:00:00'), 300.0, 5.0, ..., 280.0, 9.0,
        10.3],
       [Timestamp('2018-12-31 22:00:00'), 300.0, 11.0, ..., 277.0, 9.0,
        10.3],
       [Timestamp('2018-12-31 23:00:00'), 290.0, 12.0, ..., 277.0, 9.0,
        10.3]], dtype=object)

In [80]:
#MLP
epochs = 20
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)
mlp_model = Sequential()
mlp_model.add(Dense(12,activation = 'relu',input_dim = x_train.shape[1]))
mlp_model.add(Dense(1))
mlp_model.compile(loss = 'mse', optimizer = adam)
mlp_model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 13        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [81]:
mlp_history = mlp_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, verbose=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

In [ ]:
#wind_data['Season'] = []

